# 04.00 Data analytics
SQL can be also performed for essential data analytics. For this, we'll be using the on-time performance dataset for flights in the US from the BTS.

You can find a subset of attributes for 2018 under `/course/cs0060/data/otp_flights.tar.gz`

I.e. use `scp <your user>@ssh.cs.brown.edu:/course/cs0060/data/otp_flights.tar.gz .` to fetch the data

The first step is to load the actual data into PostgreSQL, this can be done using the `COPY` command

In [1]:
!mkdir -p data && tar xf otp_flights.tar.gz -C data/

In [2]:
!ls data

otp_flights_2018_1.csv  otp_flights_2018_2.csv  otp_flights_2018_6.csv
otp_flights_2018_10.csv otp_flights_2018_3.csv  otp_flights_2018_7.csv
otp_flights_2018_11.csv otp_flights_2018_4.csv  otp_flights_2018_8.csv
otp_flights_2018_12.csv otp_flights_2018_5.csv  otp_flights_2018_9.csv


In [3]:
!head data/otp_flights_2018_1.csv

OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,FL_DATE,ORIGIN_CITY_NAME,DEST_CITY_NAME,DISTANCE,DEP_DELAY,ARR_DELAY
UA,369,2018-01-27,"Fort Lauderdale, FL","Houston, TX",966.0,-13.0,-12.0
UA,368,2018-01-27,"Seattle, WA","San Francisco, CA",679.0,-4.0,-18.0
UA,367,2018-01-27,"Washington, DC","Houston, TX",1208.0,-2.0,1.0
UA,366,2018-01-27,"Los Angeles, CA","Chicago, IL",1744.0,-9.0,-8.0
UA,365,2018-01-27,"Jacksonville, FL","Newark, NJ",820.0,-14.0,-24.0
UA,364,2018-01-27,"Houston, TX","Phoenix, AZ",1009.0,-7.0,-19.0
UA,363,2018-01-27,"Newark, NJ","Honolulu, HI",4962.0,27.0,19.0
UA,362,2018-01-27,"Honolulu, HI","Newark, NJ",4962.0,8.0,-23.0
UA,361,2018-01-27,"Las Vegas, NV","San Francisco, CA",414.0,-5.0,-22.0


In [4]:
!psql -c 'CREATE TABLE flights(OP_UNIQUE_CARRIER VARCHAR, \
OP_CARRIER_FL_NUM NUMERIC, \
FL_DATE VARCHAR, \
ORIGIN_CITY_NAME VARCHAR, \
DEST_CITY_NAME VARCHAR, \
DISTANCE NUMERIC, \
DEP_DELAY NUMERIC, \
ARR_DELAY NUMERIC);' cs6

CREATE TABLE


In [5]:
!psql -c '\d flights' cs6

                         Table "public.flights"
      Column       |       Type        | Collation | Nullable | Default 
-------------------+-------------------+-----------+----------+---------
 op_unique_carrier | character varying |           |          | 
 op_carrier_fl_num | numeric           |           |          | 
 fl_date           | character varying |           |          | 
 origin_city_name  | character varying |           |          | 
 dest_city_name    | character varying |           |          | 
 distance          | numeric           |           |          | 
 dep_delay         | numeric           |           |          | 
 arr_delay         | numeric           |           |          | 



Loading the data into the database via `COPY FROM`

In [6]:
!psql -c "COPY flights(OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,FL_DATE, \
ORIGIN_CITY_NAME,DEST_CITY_NAME,DISTANCE,DEP_DELAY,ARR_DELAY) \
FROM '$(pwd)/data/otp_flights_2018_12.csv' DELIMITER ',' CSV HEADER;" cs6

COPY 578747


In [7]:
!psql -c 'SELECT * FROM flights LIMIT 5;' cs6

 op_unique_carrier | op_carrier_fl_num |  fl_date   | origin_city_name |         dest_city_name         | distance | dep_delay | arr_delay 
-------------------+-------------------+------------+------------------+--------------------------------+----------+-----------+-----------
 9E                |              3280 | 2018-11-01 | Atlanta, GA      | New Bern/Morehead/Beaufort, NC |    433.0 |      -5.0 |      -6.0
 9E                |              3280 | 2018-11-02 | Atlanta, GA      | New Bern/Morehead/Beaufort, NC |    433.0 |      -3.0 |      -9.0
 9E                |              3280 | 2018-11-03 | Atlanta, GA      | New Bern/Morehead/Beaufort, NC |    433.0 |     -12.0 |     -17.0
 9E                |              3280 | 2018-11-04 | Atlanta, GA      | White Plains, NY               |    780.0 |      -2.0 |     -12.0
 9E                |              3280 | 2018-11-05 | Atlanta, GA      | White Plains, NY               |    780.0 |       0.0 |       7.0
(5 rows)



## 04.01 Basic analytics
We can use simple aggregates to get information.

Aggregates that can be used are

- `AVG` computes average
- `SUM` computes sum
- `MIN` or `MAX`
- ...

A complete list is available under https://www.postgresql.org/docs/9.5/functions-aggregate.html

I.e. what is the longest flight recorded in the dataset?

In [8]:
!psql -c 'SELECT MAX(distance) FROM flights;' cs6

  max   
--------
 4983.0
(1 row)



We can now get the rows which belong to the longest flight

In [9]:
!psql -c 'SElECT origin_city_name, dest_city_name FROM flights WHERE distance=4983.0' cs6

 origin_city_name | dest_city_name 
------------------+----------------
 New York, NY     | Honolulu, HI
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Honolulu, HI     | New York, NY
 Hon

As we can see, the output is quite long because all pairs where returned!
==> we can shorten it by using the DISTINCT keyword to eliminate the duplicates

In [10]:
!psql -c 'SElECT DISTINCT origin_city_name, dest_city_name FROM flights WHERE distance=4983.0' cs6

 origin_city_name | dest_city_name 
------------------+----------------
 Honolulu, HI     | New York, NY
 New York, NY     | Honolulu, HI
(2 rows)



## 04.02 Joining other datasets
Sometimes there is information stored in other tables which we would like to combine with the current data.

For this, let's ask the following question:

Which carrier serves the most flights from New York? 

In [11]:
!psql -c "SELECT op_unique_carrier, COUNT(*) FROM flights WHERE origin_city_name \
LIKE '%New York%' GROUP BY op_unique_carrier ORDER BY COUNT(*) DESC;" cs6

 op_unique_carrier | count 
-------------------+-------
 DL                |  4304
 B6                |  3950
 9E                |  3643
 AA                |  3116
 YX                |  2823
 MQ                |  1608
 OO                |  1166
 WN                |   982
 UA                |   790
 AS                |   408
 NK                |   321
 OH                |   223
 EV                |   159
 YV                |   139
 F9                |    87
 HA                |    29
(16 rows)



The carrier code here however is unreadable, but there is a lookup table which we can join in!

In [12]:
!head airlines.csv

In [13]:
!psql -c 'CREATE TABLE airlines(code VARCHAR, name VARCHAR);' cs6

CREATE TABLE


In [14]:
!psql -c "COPY airlines(code, name) FROM '$(pwd)/airlines.csv' DELIMITER ',' CSV HEADER;" cs6

COPY 1665


Let's now join this table with the flights!

In [15]:
!psql -c "SELECT DISTINCT a.name, f.origin_city_name, f.dest_city_name \
FROM flights f JOIN airlines a ON f.op_unique_carrier = a.code WHERE f.origin_city_name LIKE '%Providence%'" cs6

          name           | origin_city_name |   dest_city_name    
-------------------------+------------------+---------------------
 Allegiant Air           | Providence, RI   | Punta Gorda, FL
 Allegiant Air           | Providence, RI   | St. Petersburg, FL
 American Airlines Inc.  | Providence, RI   | Charlotte, NC
 American Airlines Inc.  | Providence, RI   | Philadelphia, PA
 Delta Air Lines Inc.    | Providence, RI   | Atlanta, GA
 Delta Air Lines Inc.    | Providence, RI   | Detroit, MI
 Endeavor Air Inc.       | Providence, RI   | Detroit, MI
 Envoy Air               | Providence, RI   | Chicago, IL
 ExpressJet Airlines LLC | Providence, RI   | Newark, NJ
 Frontier Airlines Inc.  | Providence, RI   | Charlotte, NC
 Frontier Airlines Inc.  | Providence, RI   | Denver, CO
 Frontier Airlines Inc.  | Providence, RI   | Fort Myers, FL
 Frontier Airlines Inc.  | Providence, RI   | Myrtle Beach, SC
 Frontier Airlines Inc.  | Providence, RI   | Orlando, FL
 Frontier Airlines Inc.  | P

In [16]:
!psql -c "SELECT a.name, COUNT(*) FROM flights f JOIN airlines a ON f.op_unique_carrier = a.code \
WHERE origin_city_name LIKE '%New York%' \
GROUP BY a.name ORDER BY COUNT(*) DESC;" cs6

          name           | count 
-------------------------+-------
 Delta Air Lines Inc.    |  4304
 JetBlue Airways         |  3950
 Endeavor Air Inc.       |  3643
 American Airlines Inc.  |  3116
 Republic Airline        |  2823
 Envoy Air               |  1608
 SkyWest Airlines Inc.   |  1166
 Southwest Airlines Co.  |   982
 United Air Lines Inc.   |   790
 Alaska Airlines Inc.    |   408
 Spirit Air Lines        |   321
 PSA Airlines Inc.       |   223
 ExpressJet Airlines LLC |   159
 Mesa Airlines Inc.      |   139
 Frontier Airlines Inc.  |    87
 Hawaiian Airlines Inc.  |    29
(16 rows)



==> I.e. Delta Air Lines seems to operate the most flights out of NYC!

How about Pvd?

In [17]:
!psql -c "SELECT a.name, COUNT(*) FROM flights f JOIN airlines a ON f.op_unique_carrier = a.code \
WHERE origin_city_name LIKE '%Providence%' \
GROUP BY a.name ORDER BY COUNT(*) DESC;" cs6

          name           | count 
-------------------------+-------
 Southwest Airlines Co.  |   541
 American Airlines Inc.  |   276
 PSA Airlines Inc.       |   159
 Delta Air Lines Inc.    |   111
 Republic Airline        |   104
 Mesa Airlines Inc.      |    88
 JetBlue Airways         |    87
 Frontier Airlines Inc.  |    60
 Envoy Air               |    52
 United Air Lines Inc.   |    30
 ExpressJet Airlines LLC |    27
 Endeavor Air Inc.       |    22
 SkyWest Airlines Inc.   |    20
 Allegiant Air           |    17
(14 rows)



To reset this notebook's db, run

In [18]:
!psql -c "DROP TABLE airlines; DROP TABLE flights;" cs6

DROP TABLE


## 04.03 Data analytics in MongoDB

MongoDB also provides data aggregation features, however it's a bit more complicated than writing SQL-statements. Rather, to use aggregation across documents you'll need to define in MongoDB an aggregation pipeline:

https://docs.mongodb.com/manual/aggregation/

In [19]:
import pymongo

client = pymongo.MongoClient()

db = client['cs6']

Let's load the flight data into MongoDB

In [20]:
import csv

In [25]:
%%time
with open('data/otp_flights_2018_1.csv') as fp:
    reader = csv.DictReader(fp)
    
    rows = [dict(row) for row in reader]
    
    db.flights.insert_many(rows)

CPU times: user 10.1 s, sys: 416 ms, total: 10.5 s
Wall time: 13.2 s


In [27]:
db.flights.find_one()

{'_id': ObjectId('5dd4402bfa615e770ac935f2'),
 'OP_UNIQUE_CARRIER': 'UA',
 'OP_CARRIER_FL_NUM': '369',
 'FL_DATE': '2018-01-27',
 'ORIGIN_CITY_NAME': 'Fort Lauderdale, FL',
 'DEST_CITY_NAME': 'Houston, TX',
 'DISTANCE': '966.0',
 'DEP_DELAY': '-13.0',
 'ARR_DELAY': '-12.0'}

Let's try to do the same query above for New York in MongoDB!

In [34]:
db.flights.find_one({'ORIGIN_CITY_NAME' : {'$regex' : 'New York'}})

{'_id': ObjectId('5dd4402bfa615e770ac93688'),
 'OP_UNIQUE_CARRIER': 'UA',
 'OP_CARRIER_FL_NUM': '222',
 'FL_DATE': '2018-01-27',
 'ORIGIN_CITY_NAME': 'New York, NY',
 'DEST_CITY_NAME': 'Chicago, IL',
 'DISTANCE': '733.0',
 'DEP_DELAY': '67.0',
 'ARR_DELAY': '44.0'}

First step is to define a simple counting pipeline, which restricts documents to the ones belonging to flights originating in New York

In [41]:
res = db.flights.aggregate([{'$match' : {'ORIGIN_CITY_NAME' : {'$regex' : 'New York'}}},
                    {'$group' : {'_id' : '$OP_UNIQUE_CARRIER', 'total': { '$sum' : 1}}}])

list(res)

[{'_id': 'YX', 'total': 2702},
 {'_id': 'WN', 'total': 857},
 {'_id': 'VX', 'total': 337},
 {'_id': 'DL', 'total': 3949},
 {'_id': 'OH', 'total': 184},
 {'_id': 'F9', 'total': 86},
 {'_id': 'OO', 'total': 249},
 {'_id': 'MQ', 'total': 1102},
 {'_id': 'YV', 'total': 118},
 {'_id': 'EV', 'total': 1097},
 {'_id': 'B6', 'total': 3630},
 {'_id': 'NK', 'total': 325},
 {'_id': 'AA', 'total': 2951},
 {'_id': 'HA', 'total': 30},
 {'_id': 'AS', 'total': 58},
 {'_id': '9E', 'total': 3627},
 {'_id': 'UA', 'total': 623}]

Second step is to sort the result

In [43]:
res = db.flights.aggregate([{'$match' : {'ORIGIN_CITY_NAME' : {'$regex' : 'New York'}}},
                    {'$group' : {'_id' : '$OP_UNIQUE_CARRIER', 'total': { '$sum' : 1}}},
                           {'$sort' : {'total' : -1}}]) # 1 for ascending, -1 for descending

list(res)

[{'_id': 'DL', 'total': 3949},
 {'_id': 'B6', 'total': 3630},
 {'_id': '9E', 'total': 3627},
 {'_id': 'AA', 'total': 2951},
 {'_id': 'YX', 'total': 2702},
 {'_id': 'MQ', 'total': 1102},
 {'_id': 'EV', 'total': 1097},
 {'_id': 'WN', 'total': 857},
 {'_id': 'UA', 'total': 623},
 {'_id': 'VX', 'total': 337},
 {'_id': 'NK', 'total': 325},
 {'_id': 'OO', 'total': 249},
 {'_id': 'OH', 'total': 184},
 {'_id': 'YV', 'total': 118},
 {'_id': 'F9', 'total': 86},
 {'_id': 'AS', 'total': 58},
 {'_id': 'HA', 'total': 30}]

The `_id` field looks rather unpleasant, but it can be renamed

In [48]:
res = db.flights.aggregate([{'$match' : {'ORIGIN_CITY_NAME' : {'$regex' : 'New York'}}},
                    {'$group' : {'_id' : '$OP_UNIQUE_CARRIER', 'total': { '$sum' : 1}}},
                           {'$sort' : {'total' : -1}},
                           {'$project' : {'_id' : 0, 'carrier_code' : '$_id', 'total' : 1}}]) # 1 for ascending, -1 for descending

list(res)

[{'total': 3949, 'carrier_code': 'DL'},
 {'total': 3630, 'carrier_code': 'B6'},
 {'total': 3627, 'carrier_code': '9E'},
 {'total': 2951, 'carrier_code': 'AA'},
 {'total': 2702, 'carrier_code': 'YX'},
 {'total': 1102, 'carrier_code': 'MQ'},
 {'total': 1097, 'carrier_code': 'EV'},
 {'total': 857, 'carrier_code': 'WN'},
 {'total': 623, 'carrier_code': 'UA'},
 {'total': 337, 'carrier_code': 'VX'},
 {'total': 325, 'carrier_code': 'NK'},
 {'total': 249, 'carrier_code': 'OO'},
 {'total': 184, 'carrier_code': 'OH'},
 {'total': 118, 'carrier_code': 'YV'},
 {'total': 86, 'carrier_code': 'F9'},
 {'total': 58, 'carrier_code': 'AS'},
 {'total': 30, 'carrier_code': 'HA'}]

What is missing though is the lookup on the airline name. This can be also done in MongoDB!

In [47]:
%%time
with open('airlines.csv') as fp:
    reader = csv.DictReader(fp)
    
    rows = [dict(row) for row in reader]
    
    db.airlines.insert_many(rows)

CPU times: user 148 ms, sys: 66 ms, total: 214 ms
Wall time: 405 ms


In [84]:
res = db.flights.aggregate([{'$match' : {'ORIGIN_CITY_NAME' : {'$regex' : 'New York'}}},
                    {'$group' : {'_id' : '$OP_UNIQUE_CARRIER', 'total': { '$sum' : 1}}},
                           {'$sort' : {'total' : -1}},
                           {'$project' : {'_id' : 0, 'carrier_code' : '$_id', 'total' : 1}},
                           {'$lookup' : {'from' : 'airlines', 
                                         'localField' : 'carrier_code',
                                         'foreignField' : 'Code',
                                         'as' : 'airline'}}]) # 1 for ascending, -1 for descending

list(res)[:5]

[{'total': 3949,
  'carrier_code': 'DL',
  'airline': [{'_id': ObjectId('5dd442b0fa615e770ad1a31d'),
    'Code': 'DL',
    'Description': 'Delta Air Lines Inc.'}]},
 {'total': 3630,
  'carrier_code': 'B6',
  'airline': [{'_id': ObjectId('5dd442b0fa615e770ad1a264'),
    'Code': 'B6',
    'Description': 'JetBlue Airways'}]},
 {'total': 3627,
  'carrier_code': '9E',
  'airline': [{'_id': ObjectId('5dd442b0fa615e770ad1a1ac'),
    'Code': '9E',
    'Description': 'Endeavor Air Inc.'}]},
 {'total': 2951,
  'carrier_code': 'AA',
  'airline': [{'_id': ObjectId('5dd442b0fa615e770ad1a1bc'),
    'Code': 'AA',
    'Description': 'American Airlines Inc.'}]},
 {'total': 2702,
  'carrier_code': 'YX',
  'airline': [{'_id': ObjectId('5dd442b0fa615e770ad1a76c'),
    'Code': 'YX',
    'Description': 'Republic Airline'}]}]

Again, some projection is needed to get things nicely formatted!

In [83]:
res = db.flights.aggregate([{'$match' : {'ORIGIN_CITY_NAME' : {'$regex' : 'New York'}}},
                    {'$group' : {'_id' : '$OP_UNIQUE_CARRIER', 'total': { '$sum' : 1}}},
                           {'$sort' : {'total' : -1}},
                           {'$project' : {'_id' : 0, 'carrier_code' : '$_id', 'total' : 1}},
                           {'$lookup' : {'from' : 'airlines', 
                                         'localField' : 'carrier_code',
                                         'foreignField' : 'Code',
                                         'as' : 'airline'}},
                           {'$project' : {'total' : 1, 'airline' : {
                               '$arrayElemAt': [ '$airline.Description', 0 ] }
                                         }
                           }])

list(res)

[{'total': 3949, 'airline': 'Delta Air Lines Inc.'},
 {'total': 3630, 'airline': 'JetBlue Airways'},
 {'total': 3627, 'airline': 'Endeavor Air Inc.'},
 {'total': 2951, 'airline': 'American Airlines Inc.'},
 {'total': 2702, 'airline': 'Republic Airline'},
 {'total': 1102, 'airline': 'Envoy Air'},
 {'total': 1097, 'airline': 'ExpressJet Airlines LLC'},
 {'total': 857, 'airline': 'Southwest Airlines Co.'},
 {'total': 623, 'airline': 'United Air Lines Inc.'},
 {'total': 337, 'airline': 'Virgin America'},
 {'total': 325, 'airline': 'Spirit Air Lines'},
 {'total': 249, 'airline': 'SkyWest Airlines Inc.'},
 {'total': 184, 'airline': 'PSA Airlines Inc.'},
 {'total': 118, 'airline': 'Mesa Airlines Inc.'},
 {'total': 86, 'airline': 'Frontier Airlines Inc.'},
 {'total': 58, 'airline': 'Alaska Airlines Inc.'},
 {'total': 30, 'airline': 'Hawaiian Airlines Inc.'}]

To reset the mongodb collection, use

In [85]:
db.flights.drop()
db.airlines.drop()